In [1]:
import torch

In [ ]:
#'5fe0e8525c745b4a450533184c69e47e' data/pennchar
#'e4b3ed728d0b973ba43f49cb1e1d45f6' data/enwik8

In [2]:
loaded_corpus2 = torch.load("corpus.e4b3ed728d0b973ba43f49cb1e1d45f6.data") #enwik for training init model

In [2]:
loaded_corpus = torch.load("corpus.78a475e0154e6bad4f1db2bc8683d638.data") #for finetuning on e2e

In [14]:
loaded_corpus.dictionary.total

11876272

In [15]:
loaded_corpus2.dictionary.total

100000003

In [7]:
loaded_corpus.dictionary.total

100000003

In [8]:
loaded_corpus.dictionary.total//32

3125000

In [3]:
def tokenize_str(corpus,string):
    ids = torch.LongTensor(len(string)+5)
    token = 0
    words = list(string) + list('<GEN>')
    for word in words:
        ids[token] = corpus.dictionary.word2idx[word]
        token += 1
    return ids

def tokenize_str_enwik(corpus,string):
    ids = torch.LongTensor(len(string)+5)
    token = 0
    words = list(string) + list('<GEN>')
    for word in words:
        word = str(ord(word))
        ids[token] = corpus.dictionary.word2idx[word]
        token += 1
    return ids

In [4]:
with open("16706147797275913_ENWIK8_tune_e2e.pt","rb") as f:
    model, criterion, optimizer = torch.load(f)

In [11]:
model

RNNModel(
  (lockdrop): LockedDropout()
  (idrop): Dropout(p=0.1, inplace=False)
  (hdrop): Dropout(p=0.1, inplace=False)
  (drop): Dropout(p=0.4, inplace=False)
  (encoder): Embedding(206, 400)
  (rnns): ModuleList(
    (0): WeightDrop(
      (module): LSTM(400, 1840)
    )
    (1): WeightDrop(
      (module): LSTM(1840, 1840)
    )
    (2): WeightDrop(
      (module): LSTM(1840, 400)
    )
  )
  (decoder): Linear(in_features=1840, out_features=206, bias=True)
)

In [5]:
model.eval()

RNNModel(
  (lockdrop): LockedDropout()
  (idrop): Dropout(p=0.1, inplace=False)
  (hdrop): Dropout(p=0.1, inplace=False)
  (drop): Dropout(p=0.4, inplace=False)
  (encoder): Embedding(206, 400)
  (rnns): ModuleList(
    (0): WeightDrop(
      (module): LSTM(400, 1840)
    )
    (1): WeightDrop(
      (module): LSTM(1840, 1840)
    )
    (2): WeightDrop(
      (module): LSTM(1840, 400)
    )
  )
  (decoder): Linear(in_features=1840, out_features=206, bias=True)
)

In [6]:
def get_hiddens(input_shape,hidden_shape,num_layers=3,device='cuda'):
    hiddens = [(torch.randn(1,1,hidden_shape).to(device),torch.randn(1,1,hidden_shape).to(device))]
    for i in range(1,num_layers-1):
        hiddens.append( (torch.randn(1,1,hidden_shape).to(device),torch.randn(1,1,hidden_shape).to(device)) )
    hiddens.append( (torch.randn(1,1,input_shape).to(device),torch.randn(1,1,input_shape).to(device)) )
    return hiddens
hiddens = get_hiddens(400,1840)

In [7]:
def detokenize(corpus,dec,return_str=True):
    char_arr = []
    for i in range(len(dec)):
        c = corpus.dictionary.idx2word[dec[i]]
        char_arr.append(c)
    if return_str:
        return "".join(char_arr)
    return char_arr

def detokenize_enwik(corpus,dec,return_str=True):
    char_arr = []
    for i in range(len(dec)):
        c = chr(int( corpus.dictionary.idx2word[dec[i]] )) if corpus.dictionary.idx2word[dec[i]]!="<eos>" else ""
        char_arr.append(c)
    if return_str:
        return "".join(char_arr)
    return char_arr

In [27]:
"".join(detokenize_enwik(loaded_corpus,decoded_seq,False))

'eme[True Spice], patType[coffee shop], prea[riverside],GEN>B'

In [16]:
detokenize_enwik(loaded_corpus2,loaded_corpus2.train[:50],True)

'<mediawiki xmlns="http://www.mediawiki.org/xml/exp'

In [97]:
loaded_corpus.dictionary.word2idx

{'a': 0,
 'e': 1,
 'r': 2,
 '_': 3,
 'b': 4,
 'n': 5,
 'k': 6,
 'o': 7,
 't': 8,
 'l': 9,
 'i': 10,
 'z': 11,
 'c': 12,
 'w': 13,
 'y': 14,
 'u': 15,
 's': 16,
 'f': 17,
 'm': 18,
 'g': 19,
 'h': 20,
 'd': 21,
 '-': 22,
 'q': 23,
 'p': 24,
 'x': 25,
 '<eos>': 26,
 '<': 27,
 '>': 28,
 'N': 29,
 'j': 30,
 'v': 31,
 '.': 32,
 "'": 33,
 '1': 34,
 '9': 35,
 '5': 36,
 '0': 37,
 '&': 38,
 '$': 39,
 '3': 40,
 '2': 41,
 '4': 42,
 '8': 43,
 '6': 44,
 '7': 45,
 '#': 46,
 '\\': 47,
 '/': 48,
 '*': 49,
 '[': 50,
 'T': 51,
 ' ': 52,
 'V': 53,
 ']': 54,
 ',': 55,
 'R': 56,
 'Â': 57,
 '£': 58,
 'C': 59,
 'Ã': 60,
 '©': 61,
 'A': 62,
 'G': 63,
 'E': 64,
 'P': 65,
 '\n': 66,
 'B': 67,
 'D': 68,
 'J': 69,
 'F': 70,
 'K': 71,
 'M': 72,
 'S': 73,
 'L': 74,
 'H': 75,
 'Y': 76,
 'W': 77,
 'I': 78,
 'O': 79,
 'Z': 80,
 'U': 81,
 ';': 82,
 ':': 83,
 '?': 84,
 'Q': 85,
 '!': 86,
 '+': 87,
 '"': 88}

In [53]:
loaded_corpus.dictionary.word2idx

{'60': 0,
 '109': 1,
 '101': 2,
 '100': 3,
 '105': 4,
 '97': 5,
 '119': 6,
 '107': 7,
 '32': 8,
 '120': 9,
 '108': 10,
 '110': 11,
 '115': 12,
 '61': 13,
 '34': 14,
 '104': 15,
 '116': 16,
 '112': 17,
 '58': 18,
 '47': 19,
 '46': 20,
 '111': 21,
 '114': 22,
 '103': 23,
 '45': 24,
 '48': 25,
 '51': 26,
 '50': 27,
 '49': 28,
 '88': 29,
 '77': 30,
 '76': 31,
 '83': 32,
 '99': 33,
 '118': 34,
 '62': 35,
 '<eos>': 36,
 '102': 37,
 '87': 38,
 '98': 39,
 '95': 40,
 '80': 41,
 '54': 42,
 '121': 43,
 '84': 44,
 '85': 45,
 '52': 46,
 '53': 47,
 '73': 48,
 '55': 49,
 '56': 50,
 '57': 51,
 '72': 52,
 '67': 53,
 '65': 54,
 '90': 55,
 '117': 56,
 '74': 57,
 '35': 58,
 '82': 59,
 '69': 60,
 '68': 61,
 '91': 62,
 '93': 63,
 '123': 64,
 '125': 65,
 '70': 66,
 '42': 67,
 '39': 68,
 '124': 69,
 '106': 70,
 '38': 71,
 '113': 72,
 '59': 73,
 '122': 74,
 '44': 75,
 '71': 76,
 '40': 77,
 '41': 78,
 '79': 79,
 '75': 80,
 '86': 81,
 '66': 82,
 '78': 83,
 '195': 84,
 '169': 85,
 '63': 86,
 '89': 87,
 '170': 88,

In [4]:
len(loaded_corpus.dictionary.word2idx)

206

In [5]:
len(loaded_corpus2.dictionary.word2idx)

205

In [6]:
loaded_corpus.dictionary.word2idx['<eos>']

36

In [7]:
loaded_corpus2.dictionary.word2idx['<eos>']

36

In [8]:
set(loaded_corpus.dictionary.idx2word)-set(loaded_corpus2.dictionary.idx2word)

{'10'}

In [9]:
chr(10)

'\n'

In [8]:
def predict_from_init_vec(model,start_vec,hiddens,input_shape=400,hidden_shape=1840,return_length=10,device='cuda',until_stop=True):
    return_str = []
    #hiddens = get_hiddens(input_shape,hidden_shape,num_layers=3,device=device)
    vec = start_vec
    for i in range(return_length):
#         new_hiddens = []
#         for l in range(num_layers):
#             output, hidden = model.rnns(vec,hiddens)
#             new_hiddens.append(hidden)
            
#         hiddens = new_hiddens
        output, hiddens = model(vec,hiddens)
        #print(output.shape)
        #print(hiddens[0][0])
        decout = model.decoder(output).argmax(-1)
        vec = decout.reshape((1,1))
        #print(vec)
        vec_num = vec.detach().cpu().numpy().flatten()[0]
        return_str.append(vec_num)
        #if vec_num == 26 and until_stop:
        if vec_num == 36:
            break
        
    return return_str

In [9]:
def predictseries_enwik(model,prompt,device='cpu',input_shape=400,hidden_shape=1840,return_length=100):
    test_prompt1 = tokenize_str_enwik(loaded_corpus,prompt).view(-1,1).to(device)
    hiddens = get_hiddens(input_shape,hidden_shape,num_layers=3,device=device)
    model.to(device)
    res,hiddens = model(test_prompt1,hiddens)
    #print(res.shape)
    decode_res = model.decoder(res).argmax(-1)[-1].view((1,1))
    #print(decode_res)
    #print([b.shape for a in hiddens for b in a])
    #print(res[-1].argmax(-1).view(1,1))
    decoded_nums = predict_from_init_vec(model,decode_res,hiddens,input_shape=400,hidden_shape=1840,return_length=return_length,device='cuda',until_stop=True)
    decoded_nums = [decode_res.detach().numpy().flatten()[0]] + decoded_nums
    return detokenize_enwik(loaded_corpus,decoded_nums)

In [16]:
test_prompts = []
test_reviews = []
with open("../gpt2/e2e_nlg/data/e2e_nlg_test.txt") as f:
    for line in f:
        prompt,review = line.strip().split("<GEN>")
        test_prompts.append(prompt)
        test_reviews.append(review)

In [21]:
predictseries_enwik(model,test_prompts[3],return_length=200)

90 36


'Blue Spice is a coffee shop that serves French food and is located in the riverside area.\n\n'

In [23]:
predictseries_enwik(model,test_prompts[3],return_length=200)

'Blue Spice is a coffee shop that serves Indian food in the riverside area. It is not family-friendly.\n'

In [24]:
predictseries_enwik(model,test_prompts[3],return_length=200)

'Blue Spice is a coffee shop that serves Indian food at a moderate price range. It is located in the riverside area.\n'

In [10]:
loaded_test_prompts = []
with open("../gpt2.txt","r") as f:
    for line in f:
        loaded_test_prompts.append(line.strip().split(":")[1].lstrip().split("|")[0].rstrip())

In [26]:
loaded_test_prompts[0]

'name[The Vaults]<GEN>'

In [28]:
test_generated = []
for i in range(len(loaded_test_prompts)):
    test_generated.append( predictseries_enwik(model,loaded_test_prompts[i],return_length=200) )
    print(i,test_generated[-1])

0 The Vaults is a cheap restaurant near The Portland Arms.

1 The Cambridge Blue is a pub that serves Indian food and has a customer rating of 5 out of 5. It is located near The Portland Arms.

2 The Eagle is a cheap restaurant that serves French food. It is located near the city centre.

3 The Golden Palace is a cheap place to eat at the city centre. It is near The Sorrento.

4 The Loch Fyne is a restaurant that serves Indian food and is located near The Rice Boat.

5 The Bibimbap House is a restaurant that serves Japanese food and is located near Clare Hall.

6 The Golden Curry is a family friendly restaurant that serves Indian food. It is located near the city centre.

7 The Wrestlers is a moderately priced restaurant located near the city centre. It is not family-friendly.

8 The Eagle is a restaurant that serves English food. It is located near the city centre.

9 The Phoenix is a restaurant that serves English food and is located near The Portland Arms.

10 Browns Cambridge is a 

In [12]:
test_generated = []
for i in range(len(loaded_test_prompts)):
    test_gens_per_prompt = []
    for j in range(10):
        test_gens_per_prompt.append( predictseries_enwik(model,loaded_test_prompts[i],return_length=200) )
    test_generated.append(test_gens_per_prompt)
    print(i,test_generated[-1])

0 ['The Vaults is a pub that serves English food and is located near the city centre.\n', 'The Vaults is a family friendly restaurant located near the city centre.\n', 'The Vaults is a pub that serves English food and is located near the city centre.\n', 'The Vaults is a family friendly restaurant located near CafÃ© Brazil.\n', 'The Vaults is a cheap place to eat at the city centre near The Portland Arms.\n', 'The Vaults is a pub that serves Indian food at a moderate price range. It is located near CafÃ© Brazil.\n', 'The Vaults is a family friendly restaurant located near The Portland Arms.\n', 'The Vaults is a cheap restaurant near The Portland Arms.\n', 'The Vaults is a cheap place to eat at the city centre. It is near The Portland Arms.\n', 'The Vaults is a family friendly restaurant located near The Portland Arms.\n']
1 ['The Cambridge Blue is a pub that serves French food and has a customer rating of 5 out of 5. It is located near The Portland Arms.\n', 'The Cambridge Blue is a pu

8 ['The Aromi is a restaurant that serves Indian food at a high price range. It is located near the city centre.\n', 'The Aromi is a restaurant that serves Chinese food. It is located near the city centre.\n', 'The Aromi is a family-friendly restaurant that serves Japanese food. It is located near the city centre.\n', 'The Aromi is a family-friendly restaurant located near the city centre.\n', 'The city centre is a cheap restaurant called Aromi that serves English food. It is located near the Aromi and is not family-friendly.\n', 'The Eagle is a cheap restaurant that serves Japanese food. It is located near the city centre. It has a customer rating of 5 out of 5.\n', 'The Aromi is a family-friendly restaurant that serves Indian food. It is located near the city centre.\n', 'The Aromi is a restaurant that serves Indian food at a high price range. It is located near the city centre.\n', 'A restaurant called Aromi is a restaurant located near the city centre. It has a customer rating of 5

16 ['The Alimentum is a restaurant that serves English food and is located near the city centre.\n', 'The Alimentum is a restaurant that serves Indian food at a moderate price range. It is located near All Bar One.\n', 'The Golden Palace is a restaurant that serves English food and is located near the city centre.\n', 'The Alimentum is a Chinese restaurant located near the city centre. It is not family-friendly.\n', 'The restaurant The Alimentum is a restaurant that serves Indian food. It is located near the city centre.\n', 'The restaurant The Alimentum is a restaurant that serves Indian food. It is located near the city centre.\n', 'The Alimentum is a family friendly restaurant that serves Indian food. It is located near the city centre.\n', 'The Green Man is a pub that serves Indian food in the city centre. It is near All Bar One.\n', 'The Alimentum is a restaurant that serves Indian food at a price range of more than Â£30. It is located near All Bar One.\n', 'The Alimentum is a res

25 ['The Olive Grove is a pub that serves Japanese food in the city centre near The Six Bells.\n', 'The Olive Grove is a pub that serves English food and is located near the city centre.\n', 'The Olive Grove is a pub that serves French food in the city centre near The Six Bells.\n', 'The Olive Grove is a pub that serves French food and is located near the city centre.\n', 'The Olive Grove is a pub that serves Indian food in the city centre near the river.\n', 'The Olive Grove is a pub that serves Japanese food in the city centre near The Six Bells.\n', 'The Olive Grove is a pub that serves English food and is located near the city centre.\n', 'The Olive Grove is a pub that serves French food and is located near the city centre.\n', 'The Olive Grove is a pub that serves Indian food in the city centre near the river.\n', 'The Olive Grove is a pub that serves Indian food in the city centre. It is near the riverside and is not family-friendly.\n']
26 ['The Twenty Two is a restaurant that s

34 ['The Midsummer House is a cheap pub that serves Indian food. It is located near the city centre.\n', 'The Plough is a pub that serves French food and is located near the city centre.\n', 'The Golden Palace is a pub that serves Indian food in the city centre. It is near the riverside.\n', 'The Eagle is a pub that serves English food and is located near the city centre.\n', 'The Phoenix is a restaurant that serves Indian food in the city centre. It is near the CafÃ© Adriatic.\n', 'The Waterman is a pub that serves Indian food in the city centre. It is near The Bakers.\n', 'The Golden Palace is a pub that serves French food and is located near The Six Bells.\n', 'The city centre is a pub called The Punter that serves English food. It is located near The Portland Arms.\n', 'The Plough is a pub that serves French food in the city centre near the CafÃ© Rouge.\n', 'The pub The Mill is a pub that serves Indian food and is located near The Six Bells.\n']
35 ['The Eagle is a child friendly p

42 ['The Golden Curry is a family friendly pub that serves Indian food. It is located near the CafÃ© Rouge.\n', 'The Plough is a pub that serves French food and is located near the city centre.\n', 'The cheap price range is less than Â£20 and is located near the city centre and has a customer rating of 5 out of 5.\n', 'The Eagle is a cheap restaurant that serves Indian food. It is located in the city centre near the CafÃ© Rouge.\n', 'The pub The Waterman is a restaurant that serves English food and is located near the city centre.\n', 'The Eagle is a cheap place to eat at the city centre. It is a family friendly place that serves French food.\n', 'The Eagle is a place that serves Japanese food in the city centre near the Burger King.\n', 'The English pub in the city centre near CafÃ© Rouge is a cheap price range and has a customer rating of 5 out of 5.\n', 'The Wildwood is a pub that serves Indian food and is located near CafÃ© Rouge.\n', 'The French restaurant The Wrestlers is a famil

51 ['The Golden Curry is a family friendly restaurant that serves French food. It is located near the city centre.\n', 'The Wrestlers is a pub that serves French food and is located near the city centre. It is not family-friendly.\n', 'The Waterman is a pub that serves French food and is located near Raja Indian Cuisine.\n', 'The Wrestlers is a pub that serves Indian food and is located near Rainbow Vegetarian CafÃ©.\n', 'The Wrestlers is a pub that serves Indian food and is located near Rainbow Vegetarian CafÃ©.\n', 'The Wrestlers is a pub that serves Indian food and is located near the city centre.\n', 'The Wrestlers is a pub that serves Indian food at a moderate price range. It is located near Rainbow Vegetarian CafÃ©.\n', 'The Wrestlers is a pub that serves Japanese food and is near Rainbow Vegetarian CafÃ©.\n', 'The Golden Curry is a family friendly restaurant that serves Indian food. It is located near the city centre.\n', 'The Golden Curry is a family friendly restaurant that se

60 ['The Chinese restaurant The Mill is located near The Six Bells.\n', 'The Golden Palace is a cheap restaurant that serves Indian food. It is located near The Six Bells.\n', 'The Dumpling Tree is a place that serves Indian food at a moderate price range. It is located near The Sorrento.\n', 'The Golden Palace is a cheap restaurant that serves English food. It is located near The Six Bells.\n', 'The Fitzbillies is a restaurant near The Six Bells.\n', 'The Dumpling Tree is a family friendly restaurant near The Six Bells.\n', 'The Golden Palace is a family friendly restaurant that serves French food. It is located near The Six Bells.\n', 'The Dumpling Tree is a place that serves Japanese food and is located near The Six Bells.\n', 'The Dumpling Tree is a place that serves English food and is located near The Sorrento.\n', 'The Chinese restaurant The Dumpling Tree is located near The Six Bells.\n']
61 ['The Rice Boat is a family friendly restaurant that serves Indian food. It is located 

68 ['The French pub The Mill is located in the city centre and is located near the river.\n', 'The Phoenix is a French restaurant located near the city centre. It has a price range of more than Â£30.\n', 'The price range is a cheap place to eat in the city centre near The Six Bells.\n', 'The Chinese restaurant The Plough is a pub that serves French food. It is located near The Bakers.\n', 'The Golden Curry is a family friendly restaurant located near the city centre.\n', 'The Golden Curry is a family friendly restaurant that serves French food. It is located near the city centre.\n', 'The Golden Curry is a family friendly restaurant that serves French food. It is located near The Bakers.\n', 'The French restaurant The Plough is located in the city centre near the river.\n', 'The French restaurant The Blue Spice is located near The Six Bells.\n', 'The French restaurant The Blue Spice is located near The Six Bells.\n']
69 ['The Golden Palace is a cheap restaurant that serves French food.

77 ['The Green Man is a place that serves Indian food in the moderate price range. It is located in the city centre.\n', 'The Cricketers is a child friendly place that serves Indian food. It is located near The Portland Arms.\n', 'The Golden Curry is a restaurant that serves Indian food and is located near the city centre.\n', 'The Phoenix is a restaurant that serves Indian food at a moderate price range. It is located near the city centre.\n', 'The Golden Curry is a restaurant that serves Chinese food in the moderate price range. It is located in the city centre.\n', 'The Green Man is a moderately priced restaurant that serves Italian food. It is located near the city centre.\n', 'The Golden Curry is a restaurant that serves Italian food and is located near the city centre.\n', 'The Golden Curry is a restaurant that serves Japanese food in the city centre near the CafÃ© Rouge.\n', 'The Chinese restaurant is located in the city centre near The Bakers.\n', 'The Mill is a pub that serves

In [31]:
with open("enwik_pretrained_lstm_generated.txt","w") as f:
    for i in range(len(test_generated)):
        f.write(test_generated[i])

In [15]:
import json
json.dumps(test_generated)

'[["The Vaults is a pub that serves English food and is located near the city centre.\\n", "The Vaults is a family friendly restaurant located near the city centre.\\n", "The Vaults is a pub that serves English food and is located near the city centre.\\n", "The Vaults is a family friendly restaurant located near Caf\\u00c3\\u00a9 Brazil.\\n", "The Vaults is a cheap place to eat at the city centre near The Portland Arms.\\n", "The Vaults is a pub that serves Indian food at a moderate price range. It is located near Caf\\u00c3\\u00a9 Brazil.\\n", "The Vaults is a family friendly restaurant located near The Portland Arms.\\n", "The Vaults is a cheap restaurant near The Portland Arms.\\n", "The Vaults is a cheap place to eat at the city centre. It is near The Portland Arms.\\n", "The Vaults is a family friendly restaurant located near The Portland Arms.\\n"], ["The Cambridge Blue is a pub that serves French food and has a customer rating of 5 out of 5. It is located near The Portland Arms

In [17]:
with open("enwik_pretrained_lstm_generated_10_per.json","w") as f:
    f.write(json.dumps(test_generated))

In [1]:
def eval_success(loaded_test_prompts,loaded_test_prompts_as_dicts,test_generated):
    num_prompts_match = 0
    total_prompts = 0
    num_fields_match = 0
    total_fields = 0
    for i in range(len(loaded_test_prompts)):
        num_matches = 0
        num_fields = 0
        for k,v in loaded_test_prompts_as_dicts[i].items():
            if v in test_generated[i]:
                num_matches += 1
            num_fields += 1
        num_fields_match += num_matches
        total_fields += num_fields
        if num_matches == num_fields:
            num_prompts_match += 1
        else:
            print(i,test_generated[i],num_matches)
        total_prompts += 1
    print("{} prompts match out of {} prompts: {}%".format(num_prompts_match,total_prompts,num_prompts_match/total_prompts))
    print("{} fields match out of {} fields: {}%".format(num_fields_match,total_fields,num_fields_match/total_fields))

In [20]:
eval_success(loaded_test_prompts,loaded_test_prompts_as_dicts,test_generated)

11 prompts match out of 630 prompts: 0.01746031746031746%
2319 fields match out of 4352 fields: 0.5328584558823529%


In [211]:
eval_success(loaded_test_prompts,loaded_test_prompts_as_dicts,test_generated)

0 prompts match out of 63000 prompts: 0.0%
512 fields match out of 435200 fields: 0.11764705882352941%


In [29]:
gpt_generated = []
with open("../gpt2/e2e_nlg/test_prompts_630_gptgenerated.txt","r") as f:
    for line in f:
        gpt_generated.append(line.strip())

In [30]:
gpt_generated

['The Blue Spice coffee shop in the city centre is a coffee shop, located in the riverside area with a coffee shop name Blue Spice, and has a customer rating of 3 out of 5.',
 'The Blue Spice coffee shop in riverside has a high customer rating, and it is not child friendly',
 'The Blue Spice coffee shop near the Crowne Plaza Hotel offers fast food and is a five star coffee shop with a high customer rating.',
 'The Blue Spice coffee shop near Burger King has a customer rating of average and is a coffee shop with cheap prices.',
 'The Blue Spice coffee shop near the Crowne Plaza Hotel offers fast food and is not family-friendly',
 "The Dumpling Tree Pub in the city centre is a pub that sells cheese. It's called Blue Spice and it's near to the city centre.",
 'The riverfront pub, The Vaults, has a pub style atmosphere, and is named Blue Spice.',
 'The Blue Spice pub near Crown Plaza Hotel has a 5 out of 5 customer rating and is child friendly',
 'The Golden Palace is a',
 'The Blue Spice 

In [31]:
eval_success(loaded_test_prompts,loaded_test_prompts_as_dicts,gpt_generated)

80 prompts match out of 630 prompts: 0.12698412698412698%
3052 fields match out of 4352 fields: 0.7012867647058824%


In [28]:
import os
os.listdir("../gpt2/e2e_nlg")

['prepare_e2e_nlg.py',
 'run_clm_e2e_nlg.sh',
 'test_clm_e2e_nlg.py',
 'test_clm_e2e_nlg.sh',
 '__MACOSX',
 'view_output.ipynb',
 'data',
 'exp',
 'transformers-4.23.1.zip',
 'transformers',
 'training_log.txt',
 'gpt2_test_outputs.txt',
 '.ipynb_checkpoints',
 'test_prompts_630.txt',
 'test_prompts_630_gptgenerated.txt']

In [33]:
eval_success(loaded_test_prompts,loaded_test_prompts_as_dicts,test_generated)

0 The Golden Palace is a coffee shop in the city centre that serves Fast food. It has a low customer rating.
<eos> 2
1 The Golden Palace is a coffee shop that serves French food in the riverside area. It has a low customer rating.
<eos> 2
2 The coffee shop near the CafÃ© Brazil is The Browns Cambridge. It is rated 5 out of 5 by customers.
<eos> 2
3 The Eagle is a coffee shop near The Bakers with a low customer rating and is not family-friendly.
<eos> 1
4 The coffee shop The Vaults is located near CafÃ© Brazil.
<eos> 1
5 The Golden Palace is a pub that serves Italian food and is located in the city centre.
<eos> 2
6 The Golden Palace is a pub that serves French food and is located in the riverside area.
<eos> 2
7 The pub near the CafÃ© Rouge is The Zizzi which is a low rated Italian restaurant that is not family friendly.
<eos> 1
8 The Eagle is a cheap pub near the Express by Holiday Inn that serves English food. It is not family-friendly.
<eos> 1
9 The pub near the CafÃ© Rouge is The Z

In [37]:
from collections import Counter
freq_prompts = Counter(loaded_test_prompts)

In [39]:
for k in freq_prompts:
    if freq_prompts[k]>1:
        print(k)

In [41]:
predictseries(model,loaded_test_prompts[0],return_length=200)

'The Golden Palace is a coffee shop in the city centre that serves French food.\n<eos>'

In [42]:
predictseries(model,loaded_test_prompts[0],return_length=200)

'The Golden Palace is a coffee shop that serves Italian food and has a low customer rating. It is located in the city centre.\n<eos>'